In [19]:
## MLB bang detection with Convolutional Neural Network
# preprocessing data from directory with audio files pre-labeled
import tensorflow as tf
import os
import numpy as np
import soundfile as sf # for opening FLAC

from tensorflow.keras import layers
from tensorflow.keras import models

# repeatibility
seed = 727
tf.random.set_seed(seed)
np.random.seed(seed)

data_directory = "data"

# MAY NEED MORE FILES IN ORDER TO ACCOUNT FOR TESTING DATASET
train_data, validation_data = tf.keras.utils.audio_dataset_from_directory(
    directory = data_directory,
    validation_split = .2,
    seed = 727,
    output_sequence_length = 480000,
    subset = 'both'
)


Found 284 files belonging to 2 classes.
Using 228 files for training.
Using 56 files for validation.


In [20]:
# gets rid of 3rd axis of audio data 
# audio is single channel therefore uses two axes 
print(train_data.element_spec)

def squeeze(audio, labels):
    audio = tf.squeeze(audio, axis = -1)
    return audio, labels

train_data = train_data.map(squeeze, tf.data.AUTOTUNE)
validation_data = validation_data.map(squeeze, tf.data.AUTOTUNE)
print(train_data.element_spec)


(TensorSpec(shape=(None, 480000, None), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int32, name=None))
(TensorSpec(shape=(None, 480000), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int32, name=None))


In [21]:
# converts audio waveform to spectrogram
def audio_to_spectrogram(audio_data):
    # spectrogram to return
    spectrogram = tf.signal.stft(audio_data, frame_length = 255, frame_step = 128)
    spectrogram = tf.abs(spectrogram)

    # format the spectrogram to be like image input for CNN
    spectrogram = spectrogram[..., tf.newaxis]
    return spectrogram

# converts all audio in a dataset to spectrograms
def data_to_spec(data):
    return data.map(
        map_func = lambda audio, label: (audio_to_spectrogram(audio), label),
        num_parallel_calls = tf.data.AUTOTUNE
    )

train_data = data_to_spec(train_data)
validation_data = data_to_spec(validation_data)

# create test dataset
test_data = validation_data.shard(num_shards = 2, index = 0)
validation_data = validation_data.shard(num_shards = 2, index = 1)

In [22]:
# spectrogram representation of the 3 labels
import matplotlib.pyplot as plt
file_path = "display" 
files = os.listdir(file_path)
file = os.path.join(file_path, files[0])
audio_data, sample_rate = sf.read(file)
time = np.arange(0, len(audio_data)) / sample_rate

spectrogram = audio_to_spectrogram(audio_data)
print('Waveform shape:', audio_data.shape)
print('Spectrogram shape:', spectrogram.shape)

Waveform shape: (480000,)
Spectrogram shape: (3749, 129, 1)


In [36]:
# training the CNN model
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras import models

# ex_spec = train_data.take(1)
# print(ex_spec.element_spec)

for ex_spec, ex_labels in train_data.take(1):
    break

labels = ["no_bang", "bang"]

# optimize data for pulling
train_data = train_data.cache().shuffle(len(train_data)).prefetch(tf.data.AUTOTUNE)
validation_data = validation_data.cache().prefetch(tf.data.AUTOTUNE)
test_data = test_data.cache().prefetch(tf.data.AUTOTUNE)



InvalidArgumentError: {{function_node __wrapped__IteratorGetNext_output_types_2_device_/job:localhost/replica:0/task:0/device:CPU:0}} Can not squeeze dim[2], expected a dimension of 1, got 2
	 [[{{node Squeeze}}]] [Op:IteratorGetNext] name: 